In [42]:
from pathlib import Path
from pydub import AudioSegment
from yt_dlp import YoutubeDL
import pandas as pd
import numpy as np
import whisper 
from stable_whisper import modify_model
import os
import requests
import json
import time
import librosa
import config as config
from pydub import AudioSegment

out_dir = "dummy/"
threshold = 10
speaker_name = "dumm"

def parse_time(timestamp):
    """
    Converts a timestamp in HH:MM:SS format to seconds.
    
    Parameters:
    - timestamp: str, time in HH:MM:SS format.
    
    Returns:
    - total_seconds: float, total time in seconds.
    """
    h, m, s = map(float, timestamp.split(':'))
    total_seconds = h * 3600 + m * 60 + s
    return total_seconds

def trim_and_overwrite_audio(input_file, start_time_str):
    """
    Trims the audio file from a given start time to the end and overwrites the original file.
    
    Parameters:
    - input_file: str, path to the input WAV file.
    - start_time_str: str, start time in HH:MM:SS format where the trim should begin.
    """
    # Parse the start time
    start_time = parse_time(start_time_str)
    
    # Load the audio file
    audio = AudioSegment.from_wav(input_file)
    
    # Calculate the start time in milliseconds
    start_time_ms = start_time * 1000
    
    # Trim the audio from start_time to the end
    trimmed_audio = audio[start_time_ms:]
    
    # Overwrite the original file with the trimmed audio
    trimmed_audio.export(input_file, format="wav")
    print(f"Audio trimmed and original file overwritten")
def clupping(a):
    grouped_segments = []
    current_group = []
    current_duration = 0.0
    a['start_diff'] = a['start'].diff().shift(-1)
    a['start_diff'] = a['start_diff'].fillna(a["total"])
    print(a)
    # Iterate over the dataframe rows
    for idx, row in a.iterrows():
        try:
            if current_duration < threshold :

                if current_duration + row['start_diff'] > threshold:
                    # Combine the text of the current group
                    combined_text = ' '.join([segment['text'] for segment in current_group])
                    print(combined_text)
                    # Create a new row for the grouped segments
                    new_row = {
                        'segment_no': [segment['segment_no'] for segment in current_group],
                        'start': current_group[0]['start'],
                        'end': current_group[-1]['end'],
                        'text': combined_text,
                        'total': current_duration,
                        'word_count': sum(segment['word_count'] for segment in current_group)
                    }
                    
                    # Add the new row to the list of grouped segments
                    grouped_segments.append(new_row)
                    
                    # Reset the current group and duration
                    current_group = []
                    current_duration = 0.0
            
            else:
                    
                    # Create a new row for the grouped segments
                    new_row = {
                        'segment_no': row["segment_no"],
                        'start': row['start'],
                        'end': row['end'],
                        'text': row["text"],
                        'total': row["total"],
                        'word_count':  row["word_count"]
                    }
                    
                    # Add the new row to the list of grouped segments
                    grouped_segments.append(new_row)
                    
                    # Reset the current group and duration
                    current_group = []
                    current_duration = 0.0
        except:
            continue
        
        # Add the current row to the group
        current_group.append(row)
        current_duration += row['start_diff']

    # If there are any remaining rows in the current group, add them as a final segment
    if current_group:
        combined_text = ' '.join([segment['text'] for segment in current_group])
        new_row = {
            'segment_no': [segment['segment_no'] for segment in current_group],
            'start': current_group[0]['start'],
            'end': current_group[-1]['end'],
            'text': combined_text,
            'total': current_duration,
            'word_count': sum(segment['word_count'] for segment in current_group)
        }
        grouped_segments.append(new_row)

    # Convert the list of grouped segments to a new dataframe
    grouped_df = pd.DataFrame(grouped_segments)

    return grouped_df


def word_count(row):
    return len(row.split())

def isMusic(path):
        y, sr = librosa.load(path, sr = None)
        splits = librosa.effects.split(y = y, frame_length = 500, top_db = 5)

        if splits.size <= 5:
                return True
        else:
                return False
        
def rem_music(d_path):
     cc = []
     data_dir = d_path
     file_paths = pd.DataFrame({
     "path": [os.path.join(data_dir, filename) for filename in os.listdir(data_dir) if filename.lower().endswith('.wav') and os.path.isfile(os.path.join(data_dir, filename))]
     })
     for index, row in file_paths.iterrows():
        try:
            if isMusic(row["path"]):
                print(f"Is Music: {isMusic(row["path"])}",row["path"])
                cc.append = row["path"]
                os.remove(row["path"])
                print("removed",row["path"])
        except:
             continue
     print(cc)
def sentence_splitter(input_file,i,aout_dir = f"{out_dir}/{speaker_name}/"):
    output_directory = aout_dir
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    model = whisper.load_model('medium.en')
    model2 = whisper.load_model('large-v2')
    # result1 = model.transcribe('/home/suryasss/Barack Obama_ Yes We Can.mp3', language='en', max_initial_timestamp=None)
    modify_model(model)
    result2 = model.transcribe(input_file, language='en') 
    a = result2.segments
    # print(a)
    data = {
    'segment_no': range(0, len(a)),
    'start': [segment.start for segment in a],
    'end': [segment.end for segment in a],
    'text': [segment.text for segment in a]
    }

    # Create DataFrame
    df = pd.DataFrame(data)


    # Load the original audio file
    original_audio = AudioSegment.from_file(input_file)

    # Initialize the final audio segment
    final_audio = AudioSegment.silent(duration=0)

    # Define the duration for non-segment to be added (1 second)
    non_segment_duration = 0.2 * 1000  # pydub works with milliseconds

    # Initialize a list to store updated segment times
    updated_times = []

    # Track the current position in the final_audio
    current_position = 0
    df["shifted_start"] = df["start"].shift(-1).fillna(df["end"])
    df["end"] = np.where((df["end"] + 0.3) >= df["shifted_start"],df["shifted_start"],df["end"] + 0.3)
    # Append the first segment
    first_segment_start_time = df.iloc[0]['start'] * 1000  # Convert to milliseconds
    first_segment_end_time = df.iloc[0]['end'] * 1000  # Convert to milliseconds
    first_segment = original_audio[first_segment_start_time:first_segment_end_time]
    final_audio += first_segment

    updated_times.append({
        'segment_no': df.iloc[0]['segment_no'],
        'start': current_position / 1000,  # Convert back to seconds
        'end': (current_position + len(first_segment)) / 1000  # Convert back to seconds
    })

    # Update current position
    current_position += len(first_segment)

    # Loop through each pair of segments
    for t in range(1, len(df)):
        prev_segment_end_time = df.iloc[t-1]['end'] * 1000  # End of previous segment
        current_segment_start_time = df.iloc[t]['start'] * 1000  # Start of current segment
        current_segment_end_time = df.iloc[t]['end'] * 1000  # End of current segment
        
        # Calculate the duration of gap to use
        gap_duration = min(non_segment_duration, current_segment_start_time - prev_segment_end_time)
        
        # Add the non-segment part
        if gap_duration > 0.5:
            non_segment = original_audio[prev_segment_end_time:prev_segment_end_time + gap_duration]
        else:
            non_segment = AudioSegment.silent(duration=0.5)
        final_audio += non_segment
        current_position += len(non_segment)
        
        # Add the current segment
        current_segment = original_audio[current_segment_start_time:current_segment_end_time]
        final_audio += current_segment
        
        # Store the updated times
        updated_times.append({
            'segment_no': df.iloc[t]['segment_no'],
            'start': current_position / 1000,  # Convert back to seconds
            'end': (current_position + len(current_segment)) / 1000  # Convert back to seconds
        })
        
        # Update current position
        current_position += len(current_segment)

    # Create a new DataFrame with updated times
    updated_df = pd.DataFrame(updated_times)
    updated_df["text"] = df["text"]
    updated_df["total"] = updated_df.end - updated_df.start
    updated_df['word_count'] = updated_df['text'].apply(word_count) 
    adf = clupping(updated_df)
    fil_df = adf[adf.total > (threshold/1.75)][adf.word_count >= 6].reset_index(drop = True)
    j = i+len(fil_df["end"])
    b = np.arange(i,j)
    fil_df["inde"] = b
    fil_df['filename'] = fil_df['inde'].apply(lambda x: f'{speaker_name}_{x:05d}.wav')
    print(fil_df.shape)
    is_music = []
    texted = []
    for index, row in fil_df.iterrows():
        end_time = row['end']
        start_time = row['start']
        duration = end_time - start_time  # Calculate duration of the chunk
        output_file = os.path.join(aout_dir, f"{speaker_name}_{i:05d}.wav")  # Output file path for each chunk
        
        chunk =  final_audio[start_time * 1000:(end_time * 1000)]
        
        # print(isMusic(chunk))
        chunk.export(output_file, format="wav")
        text = model2.transcribe(output_file,language='en')
        texted.append(text["text"])
        is_music.append(isMusic(output_file))
        i = i+1 
    fil_df["ismusic"]   = is_music 
    fil_df["texted"]    = texted
    fil_df.drop_duplicates("texted",inplace = True)
    print("after dropping duplicates",fil_df.shape)
    fil_df[["filename","text","texted","start","end","ismusic"]].to_csv(f'{out_dir}/{speaker_name}/meta_chunk_{i-1}.csv',index = False)
    return i,fil_df[["filename","texted","start","end"]]
    # return df

In [43]:
a,b = sentence_splitter("/home/suryasss/Transcription/SSA_2/target_speaker.wav",1)

/home/suryasss/.conda/envs/ssconda/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

     segment_no    start      end  \
0             0     0.00     1.08   
1             1     1.28     2.56   
2             2     2.76     4.56   
3             3     4.76     5.06   
4             4     5.26     8.12   
..          ...      ...      ...   
431         431  1174.51  1175.19   
432         432  1175.19  1176.39   
433         433  1176.39  1177.83   
434         434  1177.91  1179.01   
435         435  1179.03  1180.95   

                                                  text  total  word_count  \
0                                 I love you guys too.   1.08           5   
1                                 I grew up in a place   1.28           6   
2               a lot like this one called Middletown,   1.80           7   
3                                                Ohio.   0.30           1   
4     And that's where I learned the values of loya...   2.86           9   
..                                                 ...    ...         ...   
431             

/tmp/ipykernel_1164916/52643411.py:252: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fil_df = adf[adf.total > (threshold/1.75)][adf.word_count >= 6].reset_index(drop = True)


after dropping duplicates (129, 10)


: 

In [36]:
updated_df

,segment_no,new_start_time,new_end_time
0,0,0.00,1.73
1,1,1.93,3.53
2,2,3.53,5.95
3,3,5.95,6.90
4,4,7.10,10.44
...,...,...,...
429,429,1267.78,1268.92
430,430,1268.92,1270.52
431,431,1270.52,1271.64
432,432,1271.64,1273.30


In [24]:
import pandas as pd
from pydub import AudioSegment

# Load the CSV file into a pandas DataFrame
df = a

# Load the original audio file
original_audio = AudioSegment.from_file("/home/suryasss/Transcription/SSA_2/target_speaker.wav")

# Initialize the final audio segment
final_audio = AudioSegment.silent(duration=0)

# Define the duration for non-segment to be added (1 second)
non_segment_duration = 0.2 * 1000  # pydub works with milliseconds

# Initialize a list to store updated segment times
updated_times = []

# Track the current position in the final_audio
current_position = 0
df["shifted_start"] = df["start"].shift(-1).fillna(df["end"])
df["end"] = np.where((df["end"] + 0.2) >= df["shifted_start"],df["shifted_start"],df["end"] + 0.2)
# Append the first segment
first_segment_start_time = df.iloc[0]['start'] * 1000  # Convert to milliseconds
first_segment_end_time = df.iloc[0]['end'] * 1000  # Convert to milliseconds
first_segment = original_audio[first_segment_start_time:first_segment_end_time]
final_audio += first_segment

updated_times.append({
    'segment_no': df.iloc[0]['segment_no'],
    'new_start_time': current_position / 1000,  # Convert back to seconds
    'new_end_time': (current_position + len(first_segment)) / 1000  # Convert back to seconds
})

# Update current position
current_position += len(first_segment)

# Loop through each pair of segments
for i in range(1, len(df)):
    prev_segment_end_time = df.iloc[i-1]['end'] * 1000  # End of previous segment
    current_segment_start_time = df.iloc[i]['start'] * 1000  # Start of current segment
    current_segment_end_time = df.iloc[i]['end'] * 1000  # End of current segment
    
    # Calculate the duration of gap to use
    gap_duration = min(non_segment_duration, current_segment_start_time - prev_segment_end_time)
    
    # Add the non-segment part
    if gap_duration > 0.5:
        non_segment = original_audio[prev_segment_end_time:prev_segment_end_time + gap_duration]
    else:
        non_segment = AudioSegment.silent(duration=0.5)
    final_audio += non_segment
    current_position += len(non_segment)
    
    # Add the current segment
    current_segment = original_audio[current_segment_start_time:current_segment_end_time]
    final_audio += current_segment
    
    # Store the updated times
    updated_times.append({
        'segment_no': df.iloc[i]['segment_no'],
        'new_start_time': current_position / 1000,  # Convert back to seconds
        'new_end_time': (current_position + len(current_segment)) / 1000  # Convert back to seconds
    })
    
    # Update current position
    current_position += len(current_segment)

# Create a new DataFrame with updated times
updated_df = pd.DataFrame(updated_times)

# Export the final audio
final_audio.export("check.wav", format="wav")

# Save the updated DataFrame to a new CSV file
updated_df.to_csv('updated_segments.csv', index=False)

print(f"Updated segments saved to updated_segments.csv")

Updated segments saved to updated_segments.csv


In [21]:
df["start"].shift(-1)

0         2.20
1         3.80
2         6.22
3        10.22
4        13.56
        ...   
429    1725.40
430    1727.00
431    1728.12
432    1729.78
433        NaN
Name: start, Length: 434, dtype: float64

In [15]:
updated_df[]

,segment_no,new_start_time,new_end_time
0,0,0.00,1.23
1,1,1.73,3.16
2,2,3.33,5.28
3,3,5.75,6.20
4,4,6.70,9.71
...,...,...,...
429,429,1297.87,1299.18
430,430,1299.18,1300.93
431,431,1300.93,1302.36
432,432,1302.36,1303.95


In [41]:
import pandas as pd
from pydub import AudioSegment

# Load the CSV file into a pandas DataFrame
df = a

# Load the original audio file
original_audio = AudioSegment.from_file("/home/suryasss/Transcription/SSA/target_speaker.wav")

# Initialize the final audio segment
final_audio = AudioSegment.silent(duration=0)

# Define the duration for non-segment to be added (1 second)
non_segment_duration = 0.3 * 1000  # pydub works with milliseconds

# Append the first segment
first_segment_start_time = df.iloc[0]['start'] * 1000  # Convert to milliseconds
first_segment_end_time = df.iloc[0]['end'] * 1000  # Convert to milliseconds
final_audio += original_audio[first_segment_start_time:first_segment_end_time]

# Loop through each pair of segments
for i in range(1, len(df)):
    prev_segment_end_time = df.iloc[i-1]['end'] * 1000  # End of previous segment
    current_segment_start_time = df.iloc[i]['start'] * 1000  # Start of current segment
    current_segment_end_time = df.iloc[i]['end'] * 1000  # End of current segment
    
    # Calculate the duration of gap to use
    gap_duration = min(non_segment_duration, current_segment_start_time - prev_segment_end_time)
    
    # Add the non-segment part
    if gap_duration > 0:
        non_segment = original_audio[prev_segment_end_time:prev_segment_end_time + gap_duration]
    else:
        non_segment = AudioSegment.silent(duration=0)
    final_audio += non_segment
    
    # Add the current segment
    current_segment = original_audio[current_segment_start_time:current_segment_end_time]
    final_audio += current_segment

# Export the final audio
final_audio.export("check.wav", format="wav")

<_io.BufferedRandom name='check.wav'>

In [2]:
sentence_splitter("/home/suryasss/Transcription/SSA/target_speaker.wav",1)

/home/suryasss/.conda/envs/ssconda/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

,Segment no,start,end,text,total,word_count
0,0,0.00,0.35,"Yeah,",0.35,1
1,1,0.52,1.35,that's right.,0.83,2
2,2,1.70,2.63,"So she,",0.93,2
3,3,3.30,4.25,"when he came home drunk,",0.95,5
4,4,4.12,5.29,he passed out on the couch.,1.17,6
...,...,...,...,...,...,...
1164,1164,2491.48,2493.53,A Memoir of Family and Culture and Crisis.,2.05,8
1165,1165,2493.84,2494.35,Thank you.,0.51,2
1166,1166,2494.56,2495.07,Thank you.,0.51,2
1167,1167,2495.74,2497.69,For Uncommon Knowledge and the Hoover Institu...,1.95,7
